**SAiVR Analysis Notebook**

This notebook contains code for analysing and visualizing the results of the SAiVR study project.
Currently the focus lies in analysing the results of the three different tasks: Absolute orientation task, Relative orientation task and Pointing task.
To work with this notebook you need to have the google spreadsheet of the NBP-VR-Lab and the mat files containing the results of the subjects stored locally.

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
import scipy.io as spio
from scipy.spatial import distance
#import ezodf
from matplotlib.patches import Arrow, Circle
from PIL import Image
import itertools
import ptitprince as pt
#from __future__ import print_function
from statsmodels.compat import lzip
from statsmodels.stats.anova import AnovaRM
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
from scipy import stats

Set the correct path information for the google spreadsheet and the mat files to read them in. This has to be adapted to the individual location on your machine.

In [ ]:
# define the path leading to the folder with the mat files
taskPath = "C:/Users/mein/Desktop/Cognitive Science/Saivr/Data/TaskResults"
# define the path to the xlsx file containing the spreadsheet of the lab
calenderPath = "C:/Users/mein/Desktop/Cognitive Science/Saivr/Data/Seahaven_alingment_project.xlsx"

**Preprocessing**

In [ ]:
# import calender file, only load specified columns
df = pd.read_excel(calenderPath, usecols='A,C:G')
df["Training"] = df["Training"].str.lower()
df = df[df.Discarded != 'yes']
# only keep identifier (letter)
df["Repeated"]= df["Repeated"].str[1:]
# drop first faulty measurement of f for experimental subjects and c guy who was motion sick
df = df[df["Subject#"]!=4616]
df = df[df["Subject#"]!=1338]
# drop faulty control measurements
df = df[df["Subject#"]!=8269]
df = df[df["Subject#"]!=8815]
# identifiers of successful control particpants
ids = ["a", "b", "c", "i", "p", "k", "q"]
c_measure = df[df.Training == "belt_c"][["Subject#", "Measurement#", "Repeated", "Training"]]
# only store whether the participant was in the experimental or the control group ('e' or 'c')
c_measure["Training"] = c_measure["Training"].str[-1]
# only take successful control participants
c_measure = c_measure[c_measure['Repeated'].isin(ids)]
# get the data on all experimental participants
exp_measure = df[df.Training == "belt_e"][["Subject#", "Measurement#", "Repeated", "Training"]]
exp_measure["Training"] = exp_measure["Training"].str[-1]
# combine control and experimental subjects in one data frame
measure_df = pd.concat([c_measure, exp_measure])

In [ ]:
def mat_to_py(AlignmentPath,number):
    '''
    converts mat struct with task results into (numpy) array

    also adds extra column with information whether trial was correct or wrong
    
    conditions = ["Absolute - 3s ","Absolute - inf","Relative - 3s ","Relative - inf","Pointing 3s   ","Pointing - inf"]
    '''
    path = AlignmentPath+"/AlignmentVR_SubjNo_"+number+".mat"
    mat_contents = spio.loadmat(path)
    type_array = []
    for i,cond_1 in enumerate(["Absolute", "Relative","Pointing"]):
        for j,cond_2 in enumerate(["Trial_3s", "Trial_Inf"]):
            trials_array = []
            for line in range(len(mat_contents['Output'][0][0][cond_1][cond_2][0][0])):
                value_array = []
                for column in range(len(mat_contents['Output'][0][0][cond_1][cond_2][0][0][line][0])):
                    value = mat_contents['Output'][0][0][cond_1][cond_2][0][0][line][0][column][0][0]
                    value_array.append(value)
                # check if trial is correct(true or false
                value_array.append(value_array[-1] == value_array[-3])
                trials_array.append(value_array)

            type_array.append(trials_array)

    return np.array(type_array)

Use mat_to_py function and measures_df data frame to load the performance of all subjects on all tasks into the AllResults data frame.

In [ ]:
conditions = ["Absolute - 3s ","Absolute - inf","Relative - 3s ","Relative - inf","Pointing 3s   ","Pointing - inf"]
# get all vp_nums from measure_df data frame
vp_nums = measure_df["Subject#"].astype(str).tolist()
AllResults = np.zeros((6,len(vp_nums),36))#AllResults[condition][subjectNum][Trial]
for i,e in enumerate(vp_nums):
    try:
        m = mat_to_py(taskPath,e)
        for c in range(6):       
            condperf = []
            for t in range(36):
                condperf.append(int(m[c][t][-1]))
            AllResults[c][i] = condperf  
    except:
        print(str(e)+" Not in folder")

In [ ]:
# construct a performance data frame containing the perfromance (percentage of correct answers) of 
# each subject on each task and join this with vpN to get vp_numbers as index
performances = pd.DataFrame()
vpN = pd.DataFrame(vp_nums,columns=['vp_number']).astype(int)
for cond in range(6):
    performances[cond] = np.mean(AllResults[cond],axis=1)
performances.columns = conditions
performances = vpN.join(performances).set_index('vp_number')
#performances

In [ ]:
# check some overall stats on the performance df
performances.describe()

Use performance df and measure_df to create a new data frame containing all the information relevant for further analysis.
The final data frame to work with is AllPerformances.

In [ ]:
# merge performances and measure_df on vp_numbers
df_all = performances.merge(measure_df, left_on="vp_number", right_on="Subject#").set_index("Subject#")
#df_all.head()

In [ ]:
# construct AllPerformances frame
TaskList = ['Absolute','Absolute','Relative','Relative','Pointing','Pointing']
CondList = ['3s','inf','3s','inf','3s','inf']
AllPerformances = pd.DataFrame(columns=['Subject','Task','Condition','Performance', 'Measurement', 'Training'])
for sj in list(df_all.index):
    for i,c in enumerate(conditions):
        AllPerformances = AllPerformances.append({'Subject':df_all.loc[sj]["Repeated"],'Task':TaskList[i],'Condition':CondList[i],'Performance':df_all.loc[sj][c],'Measurement':df_all.loc[sj]["Measurement#"],'Training':df_all.loc[sj]['Training']}, ignore_index=True)

In [ ]:
# Add "TC" column as combination of Task and Condition for later plotting
AllPerformances["TC"] = AllPerformances.Task + " - " + AllPerformances.Condition
# store data of experimental and control group in seperate frames to make plotting easier
AllPerformances_e = AllPerformances[AllPerformances["Training"] == 'e']
AllPerformances_c = AllPerformances[AllPerformances["Training"] == 'c']
AllPerformances.head()

AllPerformances contains 7 columns. Subject column identifies each participant with a unique letter. These letters are only unique for when control and experimental group are considered independently. The combination of Training and Subject columns is a unique identifier for each individual subject however. Task contains the name of the task. Condition stores information on the time condition. Performance stores the actual performance for the task and time condition. The measurement column indicates which measurement of the given subject we are considering (1-4). Training stoes information on whether the subject was in the experimental or the control group ('e' or 'c') and TC contains the combination of task and condition.

**Visualizing the data**

First we have a look at boxplots displaying the performance results for the three different tasks and the two different time conditions. This is done seperately for experimental and control subjects.

In [ ]:
#group tasks
#color by time condition
fig,ax = plt.subplots(figsize=(10,7))
# add the chance level line
plt.plot([-5,10],[0.5,0.5],':',color='black', linewidth=5)
# create the boxplots
sns.boxplot(data=AllPerformances_e,hue='Condition',x='Task',y='Performance', palette=["red", "royalblue"],linewidth=2.5)
# set the aestethics
ax.set_xticklabels(['Absolute','Relative','Pointing'],fontsize=15)
ax.set_ylim((0,1))
plt.legend(fontsize=20,loc=4)
plt.title('Performance of exp. subjects in the tasks',fontsize=25)
plt.ylabel('Performance (%)',fontsize=20)
plt.yticks(np.linspace(0,1,5),np.linspace(0,100,5,dtype=int),fontsize=15)
plt.xlabel("Task",fontsize=20);
#plt.show()
#plt.savefig('Results/TaskPerformancesGrouped.png', bbox_inches='tight')

In [ ]:
#group tasks
#color by time condition
fig,ax = plt.subplots(figsize=(10,7))
plt.plot([-5,10],[0.5,0.5],':',color='black', linewidth=5)
sns.boxplot(data=AllPerformances_c,hue='Condition',x='Task',y='Performance', palette=["red", "royalblue"],linewidth=2.5)
ax.set_xticklabels(['Absolute','Relative','Pointing'],fontsize=15)
ax.set_ylim((0,1))
plt.legend(fontsize=20,loc=4)
plt.title('Performance of cont. subjects in the tasks',fontsize=25)
plt.ylabel('Performance (%)',fontsize=20)
plt.yticks(np.linspace(0,1,5),np.linspace(0,100,5,dtype=int),fontsize=15)
plt.xlabel("Task",fontsize=20);
#plt.show()
#plt.savefig('Results/TaskPerformancesGrouped.png', bbox_inches='tight')

Now we have a look at the performance for the different measurements. Again this is done seperately for each combination of task and time condition and for experimental and control subjects.

In [ ]:
sns.catplot(x='TC', y='Performance', hue='Measurement', data=AllPerformances_e, kind='bar', height=7, aspect=2)
#plt.plot([-0.45,5.4],[0.5,0.5],':',color='black', linewidth=3)
#plt.legend(fontsize=20,loc=4)
plt.title('Performance of exp. subjects in the tasks for different measurements',fontsize=25)
plt.ylabel('Performance (%)',fontsize=20)
plt.yticks(np.linspace(0,0.7,8),np.linspace(0,70,8,dtype=int),fontsize=15)
plt.xlabel("Task and Condition",fontsize=20);

In [ ]:
sns.catplot(x='TC', y='Performance', hue='Measurement', data=AllPerformances_c, kind='bar', height=7, aspect=2)
#plt.plot([-0.45,5.4],[0.5,0.5],':',color='black', linewidth=3)
#plt.legend(fontsize=20,loc=4)
plt.title('Performance of cont. subjects in the tasks for different measurements',fontsize=25)
plt.ylabel('Performance (%)',fontsize=20)
plt.yticks(np.linspace(0,0.7,8),np.linspace(0,70,8,dtype=int),fontsize=15)
plt.xlabel("Task and Condition",fontsize=20);

Next we have a look at a raincloud plot. Here we mix experimental and control subjects to visualize all the available data once.

In [ ]:
#Plotting adapted from https://peerj.com/preprints/27137v1/
ax = pt.RainCloud(data=AllPerformances,hue='Condition',x='Task',y='Performance', palette=["red", "royalblue"],bw = 0.2,
                 width_viol = .5, figsize = (10,7),pointplot = False, alpha = .85, dodge = True, move = 0.2)

ax.set_xticklabels(['Absolute','Relative','Pointing'],fontsize=15)
#ax.legend(['3s','inf'],fontsize=20,loc=1)

plt.title('Performance of Subjects in the Tasks',fontsize=25)
plt.ylabel('Performance (%)',fontsize=20)
plt.xlabel("Task",fontsize=20)
plt.yticks(np.linspace(0.25,0.75,3),np.linspace(25,75,3),fontsize=15);
#plt.show()
#plt.savefig('Results/TaskPerformancesRainCloud.png', bbox_inches='tight')

**Statistics - Dive into the data**

After having had a look at different visualizations of the data, we are going to do some more rigorous analysis. We start of investigating the performance in relation to different dependent variables, such as Measurement or Condition and Task. In case any of those variables reveals itself to have a significant influence on the performance, we perform paired t-tests to pinpoint the relevant value for which the mean performance differs from the rest.
Again we treat control and experimental subjects seperately.

We first look at the experimental group.

In [ ]:
# Anova experimental group
anovarm = AnovaRM(data=AllPerformances_e,depvar='Performance',subject='Subject',within=['Task','Condition','Measurement'])
fit = anovarm.fit()
fit.summary()

In [ ]:
# Anova exp, no k, j  -> include only participants that had all measurements completed at the time of writing (11.07.19)
# keep this and the analysis done on test frame for now and out of curiosity, but ultimately we care about AllPerformances_e
test = AllPerformances_e[AllPerformances_e['Subject']!='k']
test = test[test['Subject']!='j']
anovarm = AnovaRM(data=test,depvar='Performance',subject='Subject',within=['Task','Condition','Measurement'])
fit = anovarm.fit()
fit.summary()

There are a lot of significant variables here. We will proceed with paired t-tests.

In [ ]:
# Extract performances for each task
Abs = AllPerformances_e[AllPerformances_e['Task']=='Absolute']['Performance']
Rel = AllPerformances_e[AllPerformances_e['Task']=='Relative']['Performance']
Ptg = AllPerformances_e[AllPerformances_e['Task']=='Pointing']['Performance']
# print the reults of the paired t-tests for all possible combinations
print("Abs - Rel: "+str(stats.ttest_rel(Abs, Rel)))
print("Abs - Ptg: "+str(stats.ttest_rel(Abs, Ptg)))
print("Rel - Ptg: "+str(stats.ttest_rel(Rel, Ptg)))

In [ ]:
# Extract performances for each task
Abs = test[test['Task']=='Absolute']['Performance']
Rel = test[test['Task']=='Relative']['Performance']
Ptg = test[test['Task']=='Pointing']['Performance']
# print the reults of the paired t-tests for all possible combinations
print("Abs - Rel: "+str(stats.ttest_rel(Abs, Rel)))
print("Abs - Ptg: "+str(stats.ttest_rel(Abs, Ptg)))
print("Rel - Ptg: "+str(stats.ttest_rel(Rel, Ptg)))

In [ ]:
# Extract performances for each condition
Short = AllPerformances_e[AllPerformances_e['Condition']=='3s']['Performance']
Inf = AllPerformances_e[AllPerformances_e['Condition']=='inf']['Performance']
# print the reults of the paired t-tests for all possible combinations
print("Short - Inf: "+str(stats.ttest_rel(Short, Inf)))

In [ ]:
# Extract performances for each measurement
Measurement_1 = test[test['Measurement']==1]['Performance']
Measurement_2 = test[test['Measurement']==2]['Performance']
Measurement_3 = test[test['Measurement']==3]['Performance']
Measurement_4 = test[test['Measurement']==4]['Performance']
# print the reults of the paired t-tests for all possible combinations
print("M_1 - M_2: "+str(stats.ttest_rel(Measurement_1, Measurement_2)))
print("M_1 - M_3: "+str(stats.ttest_rel(Measurement_1, Measurement_3)))
print("M_1 - M_4: "+str(stats.ttest_rel(Measurement_1, Measurement_4)))
print("M_2 - M_3: "+str(stats.ttest_rel(Measurement_2, Measurement_3)))
print("M_2 - M_4: "+str(stats.ttest_rel(Measurement_2, Measurement_4)))
print("M_3 - M_4: "+str(stats.ttest_rel(Measurement_3, Measurement_4)))

In [ ]:
# Extract performances for each measurement
Measurement_1 = AllPerformances_e[AllPerformances_e['Measurement']==1]['Performance']
Measurement_2 = AllPerformances_e[AllPerformances_e['Measurement']==2]['Performance']
Measurement_3 = AllPerformances_e[AllPerformances_e['Measurement']==3]['Performance']
Measurement_4 = AllPerformances_e[AllPerformances_e['Measurement']==4]['Performance']
# print the reults of the paired t-tests for all possible combinations
print("M_1 - M_2: "+str(stats.ttest_rel(Measurement_1, Measurement_2)))
print("M_1 - M_3: "+str(stats.ttest_rel(Measurement_1, Measurement_3)))
print("M_1 - M_4: "+str(stats.ttest_rel(Measurement_1, Measurement_4)))
print("M_2 - M_3: "+str(stats.ttest_rel(Measurement_2, Measurement_3)))
print("M_2 - M_4: "+str(stats.ttest_rel(Measurement_2, Measurement_4)))
print("M_3 - M_4: "+str(stats.ttest_rel(Measurement_3, Measurement_4)))

In [ ]:
# Extract performances for each measurement and short time condition
M_1 = test[test['Measurement']==1]
M_2 = test[test['Measurement']==2]
M_3 = test[test['Measurement']==3]
M_4 = test[test['Measurement']==4]
# get short time condition
MC_1s = M_1[M_1['Condition']=='3s']['Performance']
MC_2s = M_2[M_2['Condition']=='3s']['Performance']
MC_3s = M_3[M_3['Condition']=='3s']['Performance']
MC_4s = M_4[M_4['Condition']=='3s']['Performance']
# get long time condition
MC_1l = M_1[M_1['Condition']=='inf']['Performance']
MC_2l = M_2[M_2['Condition']=='inf']['Performance']
MC_3l = M_3[M_3['Condition']=='inf']['Performance']
MC_4l = M_4[M_4['Condition']=='inf']['Performance']
# print the reults of the paired t-tests for all possible combinations
print("MC_1s - MC_2s: "+str(stats.ttest_rel(MC_1s, MC_2s)))
print("MC_1s - MC_3s: "+str(stats.ttest_rel(MC_1s, MC_3s)))
print("MC_1s - MC_4s: "+str(stats.ttest_rel(MC_1s, MC_4s)))
print("MC_2s - MC_3s: "+str(stats.ttest_rel(MC_2s, MC_3s)))
print("MC_2s - MC_4s: "+str(stats.ttest_rel(MC_2s, MC_4s)))
print("MC_3s - MC_4s: "+str(stats.ttest_rel(MC_3s, MC_4s)))

print("MC_1l - MC_2l: "+str(stats.ttest_rel(MC_1l, MC_2l)))
print("MC_1l - MC_3l: "+str(stats.ttest_rel(MC_1l, MC_3l)))
print("MC_1l - MC_4l: "+str(stats.ttest_rel(MC_1l, MC_4l)))
print("MC_2l - MC_3l: "+str(stats.ttest_rel(MC_2l, MC_3l)))
print("MC_2l - MC_4l: "+str(stats.ttest_rel(MC_2l, MC_4l)))
print("MC_3l - MC_4l: "+str(stats.ttest_rel(MC_3l, MC_4l)))

Now we have a look at the controls. There are only 7 fully measured control participants.

In [ ]:
# Anova control
anovarm = AnovaRM(data=AllPerformances_c,depvar='Performance',subject='Subject',within=['Task','Condition','Measurement'])
fit = anovarm.fit()
fit.summary()
# do t-tests for measurement and stuff

Since only the measurement is significant, we perform post-hoc paired t-tests to identify for which measurement the mean performance significantly differs from the rest.

In [ ]:
# Extract performances for each measurement
Measurement_1 = AllPerformances_c[AllPerformances_c['Measurement']==1]['Performance']
Measurement_2 = AllPerformances_c[AllPerformances_c['Measurement']==2]['Performance']
Measurement_3 = AllPerformances_c[AllPerformances_c['Measurement']==3]['Performance']
Measurement_4 = AllPerformances_c[AllPerformances_c['Measurement']==4]['Performance']
# print the reults of the paired t-tests for all possible combinations
print("M_1 - M_2: "+str(stats.ttest_rel(Measurement_1, Measurement_2)))
print("M_1 - M_3: "+str(stats.ttest_rel(Measurement_1, Measurement_3)))
print("M_1 - M_4: "+str(stats.ttest_rel(Measurement_1, Measurement_4)))
print("M_2 - M_3: "+str(stats.ttest_rel(Measurement_2, Measurement_3)))
print("M_2 - M_4: "+str(stats.ttest_rel(Measurement_2, Measurement_4)))
print("M_3 - M_4: "+str(stats.ttest_rel(Measurement_3, Measurement_4)))